In [82]:
import gym 
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.nn import Linear, ReLU 
import torch.nn.functional as F
from torch.autograd import Variable

In [120]:
class Actor_Crtic(torch.nn.Module):
    def __init__(self):
        super(Actor_Crtic,self).__init__()
        self.fc1 = Linear(4, 128)
        self.fc2 = Linear(128, 2)
        self.fc3 = Linear(128, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        log_prob = F.log_softmax(self.fc2(x), dim=-1)
        V = F.relu(self.fc3(x))
        return log_prob, V

In [121]:
env = gym.make('CartPole-v0')
obs = env.reset()
model = Actor_Crtic()
optimizer= torch.optim.Adam(actor.parameters(), lr=3e-2)
gamma = 0.95
steps = []
CRITIC_LOSS = []
ACTOR_LOSS = []
for episode in range(1000):
    V_list = []
    log_prob = []
    rewards = []
    for step in range(200):
        # obs_list.append(obs)
        obs = np.reshape(obs, [1,-1])
        input_actor = Variable(torch.from_numpy(obs).float())
        action_probability, V = model(input_actor)
        V_list.append(V)
        action = np.random.choice(2, p=torch.exp(action_probability)[0].detach().numpy())
        log_prob.append(action_probability[0][action])
        obs, reward, done, info = env.step(action)
        rewards.append(reward)
        if done:
            steps.append(step)
            print(f'episode {episode}, step {step}', end = '\r')
            V_list = V_list - torch.mean(V_list)
            V_list /= torch.std(V_list) 
            Actor_Loss = 0
            for a, b in zip(log_prob, V_list):
                Actor_Loss -= a*b[0]
            ACTOR_LOSS.append(Actor_Loss)
            Critic_Loss = 0
            for i in range(step):             
                delta = rewards[i] + gamma*V_list[i+1][0] - V_list[i][0]
                Critic_Loss -= delta*V_list[i][0]   
            model.zero_grad()
            CRITIC_LOSS.append(Critic_Loss)
            loss=Actor_Loss + Critic_Loss
            loss.backward()
            optimizer.step()
            #-----Update Critic End------
            obs = env.reset()
            break
    if np.mean(steps[-20:]) > 180:
        break   
mid = []
interval = 30
plt.style.use('dark_background')
for i in range(len(steps) - interval):
    mid.append(np.mean(steps[i:i+interval+1]))
plt.figure(figsize=(12,12))
plt.title('Performace of Plain REINFORCE on CartPole_V0',fontsize = 'xx-large')
plt.xlabel('Episodes', fontsize = 'xx-large')
plt.ylabel('Rewards',fontsize = 'xx-large')
x_fit = list(range(len(steps) - interval))
plt.plot(x_fit, steps[interval:], '-', c = 'gray') 
plt.plot(mid, '-', c= 'green', linewidth = 5)
plt.plot(CRITIC_LOSS, c = 'red')
plt.plot(ACTOR_LOSS, c='blue')
plt.show()


RuntimeError: size mismatch, m1: [1 x 2], m2: [128 x 2] at C:\w\1\s\tmp_conda_3.7_100118\conda\conda-bld\pytorch_1579082551706\work\aten\src\TH/generic/THTensorMath.cpp:136